In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, FPMC, GRU4RecF, BERT4Rec, FOSSIL
from recbole.model.general_recommender import BPR, FISM, DMF, FISM, ItemKNN, MultiVAE, NeuMF, SpectralCF
from recbole.model.context_aware_recommender import DSSM
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
import pandas as pd
import os

In [ ]:
DATA_PATH = './data/mind/preprocessed/'

INTER_DATA_PATH = os.path.join(DATA_PATH, "mind_large_train.inter")
ITEM_DATA_PATH = os.path.join(DATA_PATH, "mind_large_train.item")

In [ ]:
inter_df = pd.read_csv(INTER_DATA_PATH, sep='\t')

In [ ]:
inter_df.sample(5)

In [ ]:
item_df = pd.read_csv(ITEM_DATA_PATH, sep='\t')

In [ ]:
from recbole.quick_start import run_recbole
from utils.utils import load_config
cfg_path = "./configs/general/bpr.yaml"

config_dict = load_config(cfg_path)
run_recbole(
    model='BPR',
    dataset='mind_small',
    config_dict=config_dict
)

In [7]:
import gdown
import os

In [8]:
os.getcwd()

'c:\\Users\\Aleksey Ryabykin\\Documents\\GitHub\\feed-ranking'

In [2]:
from utils.utils import load_config, load_data

In [3]:
load_data("mind_small")

Creating dir data
Loading data


Retrieving folder list


Processing file 1M-48Y8oTk0p77jJ6Q_JjbteqBwo6d249 mind_small.dev.inter
Processing file 1Ti49GHmiJJmON_VjxZgSf5zsjNMwMR5S mind_small.dev.item
Processing file 1k-2vSdwjPvwINsOW8tHS3xAVzJ_Fewwo mind_small.train.inter
Processing file 1z_MPGmg65M6fTcNcq1UI34cBTW6XTLFz mind_small.train.item
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1M-48Y8oTk0p77jJ6Q_JjbteqBwo6d249
To: c:\Users\Aleksey Ryabykin\Documents\GitHub\feed-ranking\data\preprocessed\mind_small\mind_small.dev.inter
100%|██████████| 70.4M/70.4M [00:06<00:00, 11.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ti49GHmiJJmON_VjxZgSf5zsjNMwMR5S
To: c:\Users\Aleksey Ryabykin\Documents\GitHub\feed-ranking\data\preprocessed\mind_small\mind_small.dev.item
100%|██████████| 33.5M/33.5M [00:02<00:00, 11.3MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1k-2vSdwjPvwINsOW8tHS3xAVzJ_Fewwo
From (redirected): https://drive.google.com/uc?id=1k-2vSdwjPvwINsOW8tHS3xAVzJ_Fewwo&confirm=t&uuid=c16eb59d-7e3c-46b9-912f-de5cee8e34a3
To: c:\Users\Aleksey Ryabykin\Documents\GitHub\feed-ranking\data\preprocessed\mind_small\mind_small.train.inter
100%|██████████| 150M/150M [00:13<00:00, 11.2MB/s] 
Downloading...
From: https://drive.google.co

Check c:\Users\Aleksey Ryabykin\Documents\GitHub\feed-ranking\data/preprocessed/mind_small



Download completed


In [19]:
import re
from itertools import cycle

def parse_log(logname):
    # Define patterns for extracting metrics and loss
    epoch_pattern = re.compile(r"epoch (\d+) ")
    metric_pattern = re.compile(r"(\w+)@(\d+) : (\d+\.\d+)")
    topk_pattern = re.compile(r"topk = \[(\d+(?:,\d+)*)\]")

    metric_names = ["precision", "recall", "ndcg", "map"]

    last_metrics = None
    # Iterate through each line in the log
    with open(logname, "r") as f:
        for line in f:
            line = line.strip()
            topk_match = topk_pattern.search(line)
            print(line, topk_match)
            if topk_match:
                last_metrics = {
                    f"{name}@{i}": None for name in metric_names for i in []
                }
                last_metrics["train_loss"] = None
                last_metrics["valid_score"] = None
            epoch_match = epoch_pattern.search(line)
            if last_metrics:
                if epoch_match:
                    epoch_num = int(epoch_match.group(1))
                    last_metrics["epoch"] = epoch_num

                train_loss_match = re.search(r"train loss: (\d+\.\d+)", line)
                if train_loss_match:
                    last_metrics["train_loss"] = float(train_loss_match.group(1))

                for name, pos, value in re.findall(metric_pattern, line):
                    last_metrics[f"{name}@{pos}"] = float(value)

                if last_metrics["valid_score"] is not None:
                    last_pos = int(list(last_metrics.keys())[-2][-1])
                    for i, name in zip(range(last_pos + 1, 11), cycle(metric_names)):
                        last_metrics[f"{name}@{i}"] = last_metrics[f"{name}@{last_pos}"]

    return last_metrics

In [ ]:
from recbole.model.context_aware_recommender import DSSM

In [20]:
parse_log('log_example.log')

Mon 08 May 2023 17:10:45 INFO  ['experiments/general/bpr.py'] None
Mon 08 May 2023 17:10:45 INFO None
General Hyper Parameters: None
gpu_id = 0 None
use_gpu = True None
seed = 2020 None
state = INFO None
reproducibility = True None
data_path = data/preprocessed/mind_small None
checkpoint_dir = saved None
show_progress = True None
save_dataset = False None
dataset_save_path = None None
save_dataloaders = False None
dataloaders_save_path = None None
log_wandb = True None
 None
Training Hyper Parameters: None
epochs = 50 None
train_batch_size = 2048 None
learner = adam None
learning_rate = 0.001 None
train_neg_sample_args = {'uniform': 1, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0} None
eval_step = 1 None
stopping_step = 10 None
clip_grad_norm = None None
weight_decay = 0.0 None
loss_decimal_place = 4 None
 None
Evaluation Hyper Parameters: None
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'fu

In [1]:
import pandas as pd

In [4]:
df= pd.read_csv('data/preprocessed/mind_small/mind_small.train.inter', sep='\t')